In [3]:
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
# Question 1: Extract Tesla Stock Data
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
print("Tesla Stock Data:")
print(tesla_data.head())

Tesla Stock Data:
                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  


In [5]:
# Question 2: Extract Tesla Revenue Data
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Find the revenue table
tables = soup.find_all("table")
revenue_table = None
for table in tables:
    if "Tesla Quarterly Revenue" in table.text:
        revenue_table = table
        break

# Extract data
tesla_revenue = []
if revenue_table:
    rows = revenue_table.find_all("tr")[1:]  # Skip header
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 2:
            date = cols[0].text.strip()
            revenue = cols[1].text.strip().replace("$", "").replace(",", "")
            if revenue:  # Ensure revenue is not empty
                tesla_revenue.append([date, float(revenue)])
    
    # Create DataFrame
    tesla_revenue_df = pd.DataFrame(tesla_revenue, columns=["Date", "Revenue"])
    tesla_revenue_df["Date"] = pd.to_datetime(tesla_revenue_df["Date"])
    print("Tesla Revenue Data:")
    print(tesla_revenue_df.head())
else:
    print("Revenue table not found.")

Tesla Revenue Data:
        Date  Revenue
0 2025-03-31  19335.0
1 2024-12-31  25707.0
2 2024-09-30  25182.0
3 2024-06-30  25500.0
4 2024-03-31  21301.0


In [6]:
# Question 3: Extract GameStop Stock Data
gamestop = yf.Ticker("GME")
gamestop_data = gamestop.history(period="max")
gamestop_data.reset_index(inplace=True)
print("GameStop Stock Data:")
print(gamestop_data.head())

GameStop Stock Data:
                       Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620128  1.693350  1.603296  1.691667  76216000   
1 2002-02-14 00:00:00-05:00  1.712708  1.716074  1.670626  1.683251  11021600   
2 2002-02-15 00:00:00-05:00  1.683251  1.687459  1.658002  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666417  1.666417  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615921  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  


In [7]:
# Question 4: Extract GameStop Revenue Data
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Find the revenue table
tables = soup.find_all("table")
revenue_table = None
for table in tables:
    if "GameStop Quarterly Revenue" in table.text:
        revenue_table = table
        break

# Extract data
gamestop_revenue = []
if revenue_table:
    rows = revenue_table.find_all("tr")[1:]  # Skip header
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 2:
            date = cols[0].text.strip()
            revenue = cols[1].text.strip().replace("$", "").replace(",", "")
            if revenue:  # Ensure revenue is not empty
                gamestop_revenue.append([date, float(revenue)])
    
    # Create DataFrame
    gamestop_revenue_df = pd.DataFrame(gamestop_revenue, columns=["Date", "Revenue"])
    gamestop_revenue_df["Date"] = pd.to_datetime(gamestop_revenue_df["Date"])
    print("GameStop Revenue Data:")
    print(gamestop_revenue_df.head())
else:
    print("Revenue table not found.")

GameStop Revenue Data:
        Date  Revenue
0 2025-01-31   1283.0
1 2024-10-31    860.0
2 2024-07-31    798.0
3 2024-04-30    882.0
4 2024-01-31   1794.0


In [16]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
print(pio.renderers)
pio.renderers.default = "browser"

# Set renderer (try 'notebook' or 'browser' if one doesn't work)
pio.renderers.default = "notebook"

# Verify data
print("Tesla Stock Data:")
print(tesla_data.head())
print(tesla_data.columns)
print("\nTesla Revenue Data:")
print(tesla_revenue_df.head())
print(tesla_revenue_df.columns)

# Ensure correct data types
tesla_data["Date"] = pd.to_datetime(tesla_data["Date"])
tesla_revenue_df["Date"] = pd.to_datetime(tesla_revenue_df["Date"])
tesla_data["Close"] = pd.to_numeric(tesla_data["Close"], errors="coerce")
tesla_revenue_df["Revenue"] = pd.to_numeric(tesla_revenue_df["Revenue"], errors="coerce")

# Optional: Limit data to recent years
tesla_data = tesla_data[tesla_data["Date"] >= "2020-01-01"]
tesla_revenue_df = tesla_revenue_df[tesla_revenue_df["Date"] >= "2020-01-01"]

# Define plotting function
def make_graph(stock_data, revenue_data, title):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        subplot_titles=("Stock Price", "Revenue"),
                        vertical_spacing=0.1)
    
    # Stock price plot
    fig.add_trace(go.Scatter(x=stock_data["Date"], y=stock_data["Close"],
                             mode="lines", name="Stock Price"), row=1, col=1)
    
    # Revenue plot
    fig.add_trace(go.Scatter(x=revenue_data["Date"], y=revenue_data["Revenue"],
                             mode="lines", name="Revenue"), row=2, col=1)
    
    fig.update_layout(title_text=title, xaxis_title="Date", showlegend=True,
                      height=600)
    fig.update_yaxes(title_text="Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($M)", row=2, col=1)
    
    # Save as image
    fig.write_image("tesla_dashboard.png")
    fig.show()

# Call the function
make_graph(tesla_data, tesla_revenue_df, "Tesla Stock Price vs Revenue")

Renderers configuration
-----------------------
    Default renderer: 'notebook'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']

Tesla Stock Data:
                          Date       Open       High        Low      Close  \
2394 2020-01-02 00:00:00-05:00  28.299999  28.713333  28.114000  28.684000   
2395 2020-01-03 00:00:00-05:00  29.366667  30.266666  29.128000  29.534000   
2396 2020-01-06 00:00:00-05:00  29.364668  30.104000  29.333332  30.102667   
2397 2020-01-07 00:00:00-05:00  30.760000  31.441999  30.224001  31.270666   
2398 2020-01-08 00:00:00-05:00  31.580000  33.232666  31.215334  32.809334   

         Volume  Dividends  Stock Splits  
2394  142981500     

ValueError: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido


In [17]:
# Question 6: GameStop Stock and Revenue Dashboard
make_graph(gamestop_data, gamestop_revenue_df, "GameStop Stock Price vs Revenue")

ValueError: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido
